In [1]:
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
from datetime import datetime, timedelta
import pandas_ta as ta
from tqdm import tqdm

pd.options.display.max_rows=None
pd.options.display.max_columns=None


In [2]:
db_conn = sqlite3.connect("../database/hh_quant.db")

start_date = '20150101'
end_date = '20200101'
lookback_window = 180

fmt_start_date = datetime.strftime(datetime.strptime(start_date, '%Y%m%d'), '%Y-%m-%d')
fmt_end_date = datetime.strftime(datetime.strptime(end_date, '%Y%m%d'), '%Y-%m-%d')
fmt_feas_start_date =  datetime.strftime((datetime.strptime(start_date, '%Y%m%d') - timedelta(lookback_window)), '%Y-%m-%d')

print(f'start::{fmt_start_date}')
print(f'end::{fmt_end_date}')
print(f'feas_start::{fmt_feas_start_date}')


start::2015-01-01
end::2020-01-01
feas_start::2014-07-05


In [3]:
# 获取历史数据
sql = f"""
select * from hh_quant_stock_history_info where datetime between '{fmt_feas_start_date}' and '{fmt_end_date}'
"""
stock_history_df = pd.read_sql_query(sql, db_conn)

In [13]:
# 处理基础数据
full_stock_df = stock_history_df[['stock_code', 'datetime', 'open', 'close', 'high', 'low', 'volume']]

In [14]:
full_stock_df.head()

,stock_code,datetime,open,close,high,low,volume
0,000001,2014-07-07,1110.96,1105.31,1113.21,1098.54,343062
1,000001,2014-07-08,1103.06,1113.21,1113.21,1095.16,346087
2,000001,2014-07-09,1112.09,1081.61,1112.09,1073.71,587891
3,000001,2014-07-10,1082.74,1083.87,1090.64,1077.10,360258
4,000001,2014-07-11,1082.74,1083.87,1087.25,1080.48,318320


In [18]:
# 构建训练数据的Label表
full_stock_df.sort_values(['stock_code', 'datetime'])
# 计算日收益率 & 历史窗口期内的平均收益率-标准差
full_stock_df['daily_return'] = full_stock_df.groupby('stock_code')['close'].pct_change()
full_stock_df['mean_return'] = full_stock_df.groupby('stock_code')['daily_return'].transform(lambda x: x.rolling(10).mean())
full_stock_df['std_return'] = full_stock_df.groupby('stock_code')['daily_return'].transform(lambda x: x.rolling(10).std())

# 计算未来5天的收益率
full_stock_df['close_in_5_days'] = full_stock_df.groupby('stock_code')['close'].shift(-5)
full_stock_df['return_5_days'] = full_stock_df['close_in_5_days'] / full_stock_df['close'] - 1

# 构建label列
full_stock_df['label'] = 0  # 默认设置为0
full_stock_df.loc[full_stock_df['return_5_days'] > full_stock_df['mean_return'] + 2 * full_stock_df['std_return'], 'label'] = 1
full_stock_df.loc[full_stock_df['return_5_days'] < full_stock_df['mean_return'] - 2 * full_stock_df['std_return'], 'label'] = -1

full_stock_df = full_stock_df[['stock_code', 'datetime', 'open', 'close', 'high', 'low', 'volume', 'label']]

KeyError: 'Column not found: close'

In [ ]:
full_stock_df.head()

,stock_code,datetime,open,high,low,volume,label
0,000001,2014-07-07,1110.96,1113.21,1098.54,343062,0
1,000001,2014-07-08,1103.06,1113.21,1095.16,346087,0
2,000001,2014-07-09,1112.09,1112.09,1073.71,587891,0
3,000001,2014-07-10,1082.74,1090.64,1077.10,360258,0
4,000001,2014-07-11,1082.74,1087.25,1080.48,318320,0


In [ ]:
# 根据日级别数据构建相关特征
base_strategy = ta.Strategy(
    name="Basic Indicator Strategy",
    description="Simple strategy with basic indicators",
    ta=[
        {"kind": "sma", "length": 5},   # 5日简单移动平均线
        {"kind": "sma", "length": 20},  # 20日简单移动平均线
        {"kind": "ema", "length": 5},   # 5日指数移动平均线
        {"kind": "ema", "length": 20},  # 20日指数移动平均线
        {"kind": "rsi", "length": 14},  # 14日相对强弱指数
        {"kind": "macd", "fast": 10, "slow": 20},  # 移动平均收敛散度
        {"kind": "bbands", "length": 20}, # 布林带
    ]
)

def calculate_ta_indicators(stock_group, strategy):
    stock_group.sort_values(by=['datetime'])
    # 添加ta指标
    # stock_group.ta.cores = 0 # 不使用多线程的方式
    stock_group.ta.strategy(strategy)
    return stock_group

full_stock_df.groupby('stock_code').apply(lambda x: calculate_ta_indicators(x, base_strategy))

[X] Ooops!!! It's True, the series 'close' was not found in stock_code, datetime, open, high, low, volume, label
[X] Ooops!!! It's True, the series 'close' was not found in stock_code, datetime, open, high, low, volume, label
[X] Ooops!!! It's True, the series 'close' was not found in stock_code, datetime, open, high, low, volume, label
[X] Ooops!!! It's True, the series 'close' was not found in stock_code, datetime, open, high, low, volume, label
[X] Ooops!!! It's True, the series 'close' was not found in stock_code, datetime, open, high, low, volume, label
[X] Ooops!!! It's True, the series 'close' was not found in stock_code, datetime, open, high, low, volume, label
[X] Ooops!!! It's True, the series 'close' was not found in stock_code, datetime, open, high, low, volume, label
[X] Ooops!!! It's True, the series 'close' was not found in stock_code, datetime, open, high, low, volume, label
[X] Ooops!!! It's True, the series 'close' was not found in stock_code, datetime, open, high, lo

KeyboardInterrupt: 